In [ ]:
from google.colab import files
import pandas as pd

# Upload file CSV yang berisi link berita
uploaded = files.upload()

# Asumsikan nama file adalah 'detik_bpjs_articles.csv'
df_links = pd.read_csv('Antara_bpjs_word_only_articles.csv', sep=';')

# Tampilkan beberapa baris pertama untuk verifikasi
df_links.head()


Saving Antara_bpjs_word_only_articles.csv to Antara_bpjs_word_only_articles.csv


,Judul,Link,Tanggal
0,BPJS Watch ingatkan kepala daerah lindungi pek...,https://www.antaranews.com/berita/4349295/bpjs...,Kemarin 15:14
1,BPJS Kesehatan Jakpus sosialisasikan pelayanan...,https://www.antaranews.com/berita/4329499/bpjs...,12 September 2024
2,Wapres minta BPJS Ketenagakerjaan jangkau peke...,https://www.antaranews.com/berita/4327879/wapr...,12 September 2024
3,Pemkab Sidoarjo lindungi ribuan RT/RW dengan B...,https://www.antaranews.com/berita/4326687/pemk...,11 September 2024
4,Anggota DPR dorong konstituen jadi peserta BPJ...,https://www.antaranews.com/berita/4317803/angg...,8 September 2024


In [ ]:
df_links = df_links.dropna()


In [ ]:
df_links.head()

,Judul,Link,Tanggal
0,BPJS Watch ingatkan kepala daerah lindungi pek...,https://www.antaranews.com/berita/4349295/bpjs...,Kemarin 15:14
1,BPJS Kesehatan Jakpus sosialisasikan pelayanan...,https://www.antaranews.com/berita/4329499/bpjs...,12 September 2024
2,Wapres minta BPJS Ketenagakerjaan jangkau peke...,https://www.antaranews.com/berita/4327879/wapr...,12 September 2024
3,Pemkab Sidoarjo lindungi ribuan RT/RW dengan B...,https://www.antaranews.com/berita/4326687/pemk...,11 September 2024
4,Anggota DPR dorong konstituen jadi peserta BPJ...,https://www.antaranews.com/berita/4317803/angg...,8 September 2024


In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

# Step 1: Function to clean unwanted characters
def clean_text(text):
    text = text.replace('\xa0', ' ').replace('Â', '').replace('â€œ', '"').replace('â€', '"')
    text = text.replace('â€™', "'").replace('â€“', '-')  # Replace apostrophes and dashes
    text = re.sub(r'â..', '', text)  # Remove any leftover encoding artifacts like â€œ, â€˜, etc.
    return text

# Step 2: Function to extract text from each article
def extract_article_details(link):
    all_text = ""
    all_tags = []

    page_number = 1
    while True:
        # Update the link to the current page
        current_link = f"{link}/{page_number}"
        response = requests.get(current_link)

        if response.status_code != 200:
            print(f"Failed to retrieve article: {current_link}")
            break

        # Set the correct encoding explicitly to 'utf-8'
        response.encoding = 'utf-8'

        soup = BeautifulSoup(response.text, "html.parser")

        # Remove unwanted content
        for unwanted in soup.find_all(['div', 'span'], class_=['lihatjg', 'aevp_title', 'aevp', 'baca-juga']):
            unwanted.decompose()
        for unwanted_p in soup.find_all('p', class_='text-muted mt-2 small'):
            unwanted_p.decompose()

        # Title
        title_tag = soup.find('div', class_='wrap__article-detail-title')
        title = title_tag.find('h1').get_text(strip=True) if title_tag and title_tag.find('h1') else "No title"

        # Date
        date_tag = soup.find('span', class_='text-secondary')
        date = date_tag.get_text(strip=True) if date_tag else "No date"

        # Article text
        text_tag = soup.find('div', class_='wrap__article-detail-content post-content')
        text = text_tag.get_text(strip=True) if text_tag else "No text"

        # Clean the text
        text = clean_text(text)
        all_text += text + "\n"  # Add cleaned text from this page

        # Tags
        tag_container = soup.find('div', class_='blog-tags')
        if tag_container:
            tag_links = tag_container.find_all('a')
            tags = [tag.get_text(strip=True) for tag in tag_links]
            all_tags.extend(tags)

        # Check for next page
        next_page_tag = soup.find('a', class_='detail__anchor-numb', attrs={'dtr-act': 'button selanjutnya'})
        if not next_page_tag:
            break  # No more pages, exit the loop

        page_number += 1  # Move to the next page

    tags_str = ', '.join(set(all_tags)) if all_tags else "No tags"  # Unique tags

    return {
        'Judul': title,
        'Link': link,
        'Teks': all_text.strip(),
        'Tanggal': date,
        'Tags': tags_str
    }

# Step 3: Loop to extract articles from all links in the CSV file
all_articles = []
for link in df_links['Link']:
    print(f"Extracting article from {link}")

    # Check if the link contains 'video' and skip if it does
    if "video" in link:
        print(f"Skipping video link: {link}")
        continue

    try:
        article_details = extract_article_details(link)
        all_articles.append(article_details)
    except Exception as e:
        print(f"Failed to process article from {link}: {e}")

    # Delay to avoid sending requests too quickly
    time.sleep(2)

# Step 4: Convert results into a DataFrame
df_articles = pd.DataFrame(all_articles)

# Step 5: Export DataFrame to CSV with semicolon separator
df_articles.to_csv('Antar_bpjs_article_details.csv', sep=';', index=False)

print("Data artikel berhasil diekspor ke 'Antar_bpjs_article_details.csv'")


Extracting article from https://www.antaranews.com/berita/4349295/bpjs-watch-ingatkan-kepala-daerah-lindungi-pekerja-badan-ad-hoc
Extracting article from https://www.antaranews.com/berita/4329499/bpjs-kesehatan-jakpus-sosialisasikan-pelayanan-mudah-bagi-peserta-jkn
Extracting article from https://www.antaranews.com/berita/4327879/wapres-minta-bpjs-ketenagakerjaan-jangkau-pekerja-kawasan-terpencil
Extracting article from https://www.antaranews.com/berita/4326687/pemkab-sidoarjo-lindungi-ribuan-rt-rw-dengan-bpjs-ketenagakerjaan
Extracting article from https://www.antaranews.com/berita/4317803/anggota-dpr-dorong-konstituen-jadi-peserta-bpjs-ketenagakerjaan
Extracting article from https://www.antaranews.com/berita/4315687/bpjs-kesehatan-perluas-layanan-online-hingga-ke-nagari
Extracting article from https://www.antaranews.com/berita/4310307/bpjs-pemprov-babel-kerja-sama-jaminan-sosial-tenaga-kerja
Extracting article from https://www.antaranews.com/berita/4308855/bpjs-yogyakarta-gencarkan-l